<a href="https://colab.research.google.com/github/NamasteIndia/pairip-protection-remover/blob/main/Pairip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
PairIP Protection Remover v1.3 (Google Colab Optimized)
Cross-platform tool for patching Flutter applications
Adapted from: https://github.com/void-eth/pairip-protection-remover
Optimized for Google Colab with GitHub file downloads
"""
import os
import sys
import shutil
import re
import subprocess
import glob
import time
import zipfile
import urllib.request
import threading
import itertools
from pathlib import Path
from multiprocessing import Pool
from google.colab import files

# Install dependencies
!pip install colorama tqdm

from colorama import init, Fore, Style
from tqdm import tqdm

# Initialize colorama
init(autoreset=True)

class Logger:
    """Beautiful logger for terminal output"""
    def __init__(self):
        self.progress_bars = {}

    def info(self, message):
        print(f"{Fore.BLUE}[i]{Style.RESET_ALL} {message}")

    def success(self, message):
        print(f"{Fore.GREEN}[✓]{Style.RESET_ALL} {message}")

    def error(self, message):
        print(f"{Fore.RED}[✗]{Style.RESET_ALL} {message}")

    def warning(self, message):
        print(f"{Fore.YELLOW}[!]{Style.RESET_ALL} {message}")

    def header(self, message):
        print(f"\n{Style.BRIGHT}{Fore.CYAN}▶ {message}{Style.RESET_ALL}")

    def subheader(self, message):
        print(f"{Fore.CYAN}  ➤ {message}{Style.RESET_ALL}")

    def create_progress_bar(self, name, total, desc="Processing"):
        self.progress_bars[name] = tqdm(total=total,
                                        desc=f"{Fore.CYAN}{desc}{Style.RESET_ALL}",
                                        bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]",
                                        colour='cyan')
        return self.progress_bars[name]

    def update_progress(self, name, amount=1):
        if name in self.progress_bars:
            self.progress_bars[name].update(amount)

    def close_progress(self, name):
        if name in self.progress_bars:
            self.progress_bars[name].close()
            del self.progress_bars[name]

log = Logger()

def show_spinner(stop_event, message="LOADING"):
    """Show a simple spinner animation"""
    birds = itertools.cycle(["𓅰", "𓅬", "𓅭", "𓅮"])
    colors = itertools.cycle([Fore.RED, Fore.YELLOW, Fore.GREEN, Fore.CYAN])
    while not stop_event.is_set():
        bird = next(birds)
        color = next(colors)
        sys.stdout.write(f'\r{message}... {color}{bird}{Style.RESET_ALL}')
        sys.stdout.flush()
        time.sleep(0.2)
    sys.stdout.write(f'\r{Fore.GREEN}✔ Completed{Style.RESET_ALL}\n')
    sys.stdout.flush()

def run_with_spinner(cmd, verbose=True, exit_on_error=True, spinner_message="Processing"):
    """Run a command with a spinner"""
    stop_event = threading.Event()
    spinner_thread = threading.Thread(target=show_spinner, args=(stop_event, spinner_message))
    spinner_thread.start()

    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        stop_event.set()
        spinner_thread.join()

        if result.returncode != 0 and exit_on_error:
            if verbose:
                log.error(f"Error executing command: {cmd}\n{result.stderr}")
            sys.exit(1)
        return result.stdout.strip()
    except Exception as e:
        stop_event.set()
        spinner_thread.join()
        if verbose:
            log.error(f"Error executing command: {cmd}\n{str(e)}")
        if exit_on_error:
            sys.exit(1)
        raise

def extract_file(zipfile_path, target=None):
    """Extract file from zip archive"""
    try:
        with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
            if target:
                zip_ref.extract(target)
            else:
                zip_ref.extractall()
        return True
    except Exception as e:
        log.error(f"Failed to extract {target or 'files'} from {zipfile_path}: {e}")
        return False

def delete_dir_crossplatform(path):
    """Delete directory or file"""
    try:
        if os.path.isfile(path):
            os.remove(path)
        elif os.path.isdir(path):
            shutil.rmtree(path, ignore_errors=True)
        return True
    except Exception as e:
        log.warning(f"Failed to delete {path}: {e}")
        return False

def download_required_files():
    """Download required files from GitHub"""
    files_to_download = {
        "APKEditor-1.4.3.jar": "https://github.com/NamasteIndia/pairip-protection-remover/raw/main/APKEditor-1.4.3.jar",
        "uber-apk-signer.jar": "https://github.com/NamasteIndia/pairip-protection-remover/raw/main/uber-apk-signer.jar",
        "libpairipcorex.so": "https://github.com/NamasteIndia/pairip-protection-remover/raw/main/libpairipcorex.so"
    }

    log.header("Downloading required files...")
    for file_name, url in files_to_download.items():
        if not os.path.exists(file_name):
            try:
                log.info(f"Downloading {file_name}...")
                urllib.request.urlretrieve(url, file_name)
                log.success(f"Downloaded {file_name}")
            except Exception as e:
                log.error(f"Failed to download {file_name}: {e}")
                sys.exit(1)
        else:
            log.info(f"{file_name} already exists")

def process_xml_file(xml_path):
    """Process an XML file"""
    try:
        with open(xml_path, 'r', encoding='utf-8') as f:
            content = f.read()
        if '<' not in content or '>' not in content:
            return False
        return True
    except Exception as e:
        log.warning(f"Error processing {xml_path}: {e}")
        return False

def patch_files():
    """Apply patches to decompiled files"""
    base_dir = 'merged_app_decompile_xml'
    smali_base_dir = os.path.join(base_dir, 'smali')
    base_lib_dir = os.path.join(base_dir, 'root/lib')
    resources_dir = os.path.join(base_dir, 'resources')

    log.header("Applying patches to decompiled files")

    # Verify architectures
    log.subheader("Verifying native library architectures...")
    architectures = []
    try:
        with zipfile.ZipFile("merged_app.apk", 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith("lib/") and file.endswith("libpairipcore.so"):
                    arch = file.split("/")[1]
                    architectures.append(arch)
        if architectures:
            log.success(f"Found architectures: {', '.join(architectures)}")
        else:
            log.warning("No native libraries found in APK")
    except Exception as e:
        log.warning(f"Could not verify architectures: {e}")

    # Smali patching definitions
    new_clinit_block = [
        ".method static constructor <clinit>()V",
        "    .registers 1",
        "",
        "    .line 30",
        "    const-string v0, \"pairipcorex\"",
        "    invoke-static {v0}, Ljava/lang/System;->loadLibrary(Ljava/lang/String;)V",
        "",
        "    const-string v0, \"pairipcore\"",
        "    invoke-static {v0}, Ljava/lang/System;->loadLibrary(Ljava/lang/String;)V",
        "",
        "    return-void",
        ".end method"
    ]

    new_verify_signature_block = [
        ".method static verifySignatureMatches(Ljava/lang/String;)Z",
        "    .registers 1",
        "",
        "    const/4 p0, 0x1",
        "    return p0",
        ".end method"
    ]

    new_initialize_license_block = [
        ".method public initializeLicenseCheck()V",
        "    .registers 1",
        "    return-void",
        ".end method"
    ]

    new_connect_license_block = [
        ".method private connectToLicensingService()V",
        "    .registers 1",
        "    return-void",
        ".end method"
    ]

    # Apply Smali patches
    log.subheader("Patching protection code...")
    stop_event = threading.Event()
    spinner_thread = threading.Thread(target=show_spinner, args=(stop_event, "Patching Smali"))
    spinner_thread.start()

    vmrunner_patched = False
    sigcheck_patched = False
    verify_signature_patched = False
    initialize_license_patched = False
    connect_license_patched = False

    smali_dirs = [d for d in os.listdir(smali_base_dir) if d.startswith('classes') and os.path.isdir(os.path.join(smali_base_dir, d))]

    if not smali_dirs:
        stop_event.set()
        spinner_thread.join()
        log.warning("No smali/classes* directories found, skipping Smali patching")
    else:
        for smali_dir in smali_dirs:
            pairip_dir = os.path.join(smali_base_dir, smali_dir, 'com/pairip')
            if not os.path.exists(pairip_dir):
                continue

            for root, dirs, files in os.walk(pairip_dir):
                if 'VMRunner.smali' in files:
                    vmrunner_path = os.path.join(root, 'VMRunner.smali')
                    try:
                        with open(vmrunner_path, 'r', encoding='utf-8') as f:
                            lines = f.readlines()

                        new_lines = []
                        inside_clinit = False
                        method_start_found = False

                        for line in lines:
                            if not inside_clinit and line.strip().startswith('.method') and '<clinit>()V' in line:
                                inside_clinit = True
                                method_start_found = True
                                new_lines.extend(line + '\n' for line in new_clinit_block)
                                continue

                            if inside_clinit:
                                if line.strip() == '.end method':
                                    inside_clinit = False
                                continue

                            new_lines.append(line)

                        if method_start_found:
                            with open(vmrunner_path, 'w', encoding='utf-8') as f:
                                f.writelines(new_lines)
                            vmrunner_patched = True
                    except Exception as e:
                        log.error(f"Failed to patch VMRunner.smali: {e}")

                if 'SignatureCheck.smali' in files or 'LicenseClient.smali' in files:
                    smali_file = 'LicenseClient.smali' if 'LicenseClient.smali' in files else 'SignatureCheck.smali'
                    sig_path = os.path.join(root, smali_file)
                    try:
                        with open(sig_path, 'r', encoding='utf-8') as f:
                            lines = f.readlines()

                        new_lines = []
                        inside_verify = False
                        inside_verify_signature = False
                        inside_initialize_license = False
                        inside_connect_license = False
                        verify_method_found = False
                        verify_signature_method_found = False
                        initialize_license_method_found = False
                        connect_license_method_found = False

                        i = 0
                        while i < len(lines):
                            stripped = lines[i].strip()

                            if not inside_verify and stripped.startswith('.method') and 'verifyIntegrity(Landroid/content/Context;)V' in stripped:
                                inside_verify = True
                                verify_method_found = True
                                new_lines.append(lines[i])
                                i += 1
                                new_lines.append('    .registers 1\n')
                                new_lines.append('    return-void\n')
                                new_lines.append('.end method\n')
                                while i < len(lines) and not lines[i].strip().startswith('.end method'):
                                    i += 1
                                i += 1
                                continue

                            if not inside_verify_signature and stripped.startswith('.method') and 'verifySignatureMatches(Ljava/lang/String;)Z' in stripped:
                                inside_verify_signature = True
                                verify_signature_method_found = True
                                new_lines.extend(line + '\n' for line in new_verify_signature_block)
                                while i < len(lines) and not lines[i].strip().startswith('.end method'):
                                    i += 1
                                i += 1
                                continue

                            if not inside_initialize_license and stripped.startswith('.method') and 'initializeLicenseCheck()V' in stripped:
                                inside_initialize_license = True
                                initialize_license_method_found = True
                                new_lines.extend(line + '\n' for line in new_initialize_license_block)
                                while i < len(lines) and not lines[i].strip().startswith('.end method'):
                                    i += 1
                                i += 1
                                continue

                            if not inside_connect_license and stripped.startswith('.method') and 'connectToLicensingService()V' in stripped:
                                inside_connect_license = True
                                connect_license_method_found = True
                                new_lines.extend(line + '\n' for line in new_connect_license_block)
                                while i < len(lines) and not lines[i].strip().startswith('.end method'):
                                    i += 1
                                i += 1
                                continue

                            new_lines.append(lines[i])
                            i += 1

                        if verify_method_found or verify_signature_method_found or initialize_license_method_found or connect_license_method_found:
                            with open(sig_path, 'w', encoding='utf-8') as f:
                                f.writelines(new_lines)
                            if verify_method_found:
                                sigcheck_patched = True
                            if verify_signature_method_found:
                                verify_signature_patched = True
                            if initialize_license_method_found:
                                initialize_license_patched = True
                            if connect_license_method_found:
                                connect_license_patched = True
                    except Exception as e:
                        log.error(f"Failed to patch {smali_file}: {e}")

        stop_event.set()
        spinner_thread.join()

        if vmrunner_patched:
            log.success("Patched VMRunner.smali")
        if sigcheck_patched:
            log.success("Patched verifyIntegrity")
        if verify_signature_patched:
            log.success("Patched verifySignatureMatches")
        if initialize_license_patched:
            log.success("Patched initializeLicenseCheck")
        if connect_license_patched:
            log.success("Patched connectToLicensingService")
        if not (vmrunner_patched or sigcheck_patched or verify_signature_patched or initialize_license_patched or connect_license_patched):
            log.warning("No Smali files patched")

    # Copy .so Files
    log.subheader("Copying native libraries...")
    so_files_to_copy = [
        'libpairipcorex.so',
        'libFirebaseCppApp.so'
    ]

    current_dir = os.getcwd()
    missing = [f for f in so_files_to_copy if not os.path.exists(os.path.join(current_dir, f))]
    if missing:
        log.error(f"Missing .so files: {', '.join(missing)}")
        return False

    supported_archs = ['armeabi-v7a', 'arm64-v8a', 'x86', 'x86_64']
    lib_dirs_found = 0
    arch_types = []
    for root, dirs, files in os.walk(base_lib_dir):
        if 'libpairipcore.so' in files:
            arch_type = os.path.basename(root)
            if arch_type in architectures and arch_type in supported_archs:
                arch_types.append(arch_type)
                lib_dirs_found += 1
                for so_file in so_files_to_copy:
                    src_path = os.path.join(current_dir, so_file)
                    dst_path = os.path.join(root, so_file)
                    try:
                        shutil.copy2(src_path, dst_path)
                        log.success(f'Copied {so_file} to {arch_type} architecture')
                    except Exception as e:
                        log.error(f"Failed to copy {so_file} to {arch_type}: {e}")
                        return False
    if lib_dirs_found == 0:
        log.warning("No library directories found containing libpairipcore.so")
    else:
        log.success(f"Libraries copied to {lib_dirs_found} architecture(s): {', '.join(arch_types)}")

    # Patch file_paths.xml
    log.subheader("Patching file paths configuration...")
    file_paths_patched = False
    pattern = re.compile(
        r'<external-path\s+name="[^"]*"\s+path="Android/data/[^"]*/files/Pictures"\s*/>',
        re.IGNORECASE
    )
    replacement = '<external-files-path name="my_images" path="Pictures/" />'

    for root, dirs, files in os.walk(resources_dir):
        if 'file_paths.xml' in files and 'res/xml' in root:
            xml_path = os.path.join(root, 'file_paths.xml')
            try:
                with open(xml_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                if pattern.search(content):
                    content_new = pattern.sub(replacement, content)
                    with open(xml_path, 'w', encoding='utf-8') as f:
                        f.write(content_new)
                    file_paths_patched = True
                    log.success("Patched file_paths.xml")
            except Exception as e:
                log.error(f"Failed to patch {xml_path}: {e}")

    if not file_paths_patched:
        log.warning("No matching <external-path> entries found")

    return vmrunner_patched or sigcheck_patched or verify_signature_patched or initialize_license_patched or connect_license_patched or file_paths_patched

def process_apk(apks_file):
    """Process an APKS file in Colab"""
    work_dir = '/content'
    os.chdir(work_dir)

    # Install Java
    log.header("Installing OpenJDK...")
    !apt-get update -q && apt-get install -y openjdk-17-jdk
    log.success("Java installed")

    # Download required files
    download_required_files()

    base_name = os.path.splitext(os.path.basename(apks_file))[0]
    decompile_dir = "merged_app_decompile_xml"

    total_steps = 11
    progress = log.create_progress_bar("main", total_steps, "Patching process")

    # Step 1: Extract base.apk
    start_time = time.time()
    log.header("Step 1/11: Extracting base.apk")
    success = extract_file(apks_file, "base.apk")
    if not success:
        log.error("Failed to extract base.apk")
        return None
    log.success(f"Extracted base.apk in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 2: Create libFirebaseCppApp.so
    start_time = time.time()
    log.header("Step 2/11: Creating libFirebaseCppApp.so")
    if not os.path.exists("libFirebaseCppApp.so"):
        if os.path.exists("base.apk"):
            shutil.copy("base.apk", "libFirebaseCppApp.so")
            log.success("Created libFirebaseCppApp.so from base.apk")
        else:
            log.error("base.apk not found to create libFirebaseCppApp.so")
            return None
    else:
        log.success("libFirebaseCppApp.so already exists")
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 3: Remove old files
    start_time = time.time()
    log.header("Step 3/11: Cleaning previous files")
    cleaned = 0
    for f in ["merged_app.apk", decompile_dir]:
        if os.path.exists(f):
            delete_dir_crossplatform(f)
            cleaned += 1
    log.success(f"Removed {cleaned} old files/directories in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 4: Merge APKS to APK
    start_time = time.time()
    log.header("Step 4/11: Merging APKS to APK")
    run_with_spinner(
        f'java -jar APKEditor-1.4.3.jar m -i "{apks_file}" -o merged_app.apk -extractNativeLibs true',
        spinner_message="Merging APKS"
    )
    if os.path.exists("merged_app.apk"):
        merged_size = os.path.getsize("merged_app.apk") / (1024 * 1024)
        log.success(f"Successfully merged to APK (Size: {merged_size:.2f} MB) in {time.time() - start_time:.1f} seconds")
    else:
        log.error("Failed to merge APKS to APK")
        return None
    progress.update(1)

    # Step 5: Prepare decompilation
    start_time = time.time()
    log.header("Step 5/11: Preparing decompilation")
    if os.path.exists(decompile_dir):
        shutil.rmtree(decompile_dir, ignore_errors=True)
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 6: Decompile the merged APK
    start_time = time.time()
    log.header("Step 6/11: Decompiling merged APK")
    run_with_spinner(
        "java -jar APKEditor-1.4.3.jar d -i merged_app.apk",
        spinner_message="Decompiling APK"
    )
    if os.path.exists(decompile_dir):
        log.success(f"Decompilation completed in {time.time() - start_time:.1f} seconds")
    else:
        log.error("Decompilation failed")
        return None
    progress.update(1)

    # Step 7: Modify AndroidManifest.xml
    start_time = time.time()
    log.header("Step 7/11: Modifying AndroidManifest.xml")
    manifest_path = os.path.join(decompile_dir, "AndroidManifest.xml")
    entries_removed = 0
    permissions_removed = 0
    if os.path.exists(manifest_path):
        with open(manifest_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Remove license check entries
        patterns = [
            r'<activity[^>]+com\.pairip\.licensecheck\.LicenseActivity[^<]+/>',
            r'<provider[^>]+com\.pairip\.licensecheck\.LicenseContentProvider[^<]+/>'
        ]
        for pattern in patterns:
            matches = re.findall(pattern, content, flags=re.DOTALL)
            entries_removed += len(matches)
            content = re.sub(pattern, '', content, flags=re.DOTALL)

        # Remove CHECK_LICENSE permission
        permission_pattern = r'<uses-permission[^>]+android:name="com\.android\.vending\.CHECK_LICENSE"[^<]*/>'
        matches = re.findall(permission_pattern, content, flags=re.DOTALL)
        permissions_removed = len(matches)
        content = re.sub(permission_pattern, '', content, flags=re.DOTALL)

        if entries_removed > 0 or permissions_removed > 0:
            with open(manifest_path, 'w', encoding='utf-8') as f:
                f.write(content)
            if entries_removed > 0:
                log.success(f"Removed {entries_removed} license check entries")
            if permissions_removed > 0:
                log.success(f"Removed CHECK_LICENSE permission")
        else:
            log.warning("No license check entries or CHECK_LICENSE permission found")
    else:
        log.warning("AndroidManifest.xml not found")
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 8: Patch files
    start_time = time.time()
    log.header("Step 8/11: Patching decompiled files")
    patch_result = patch_files()
    if not patch_result:
        log.warning("No files were patched")
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 9: Preprocess XML files in parallel
    start_time = time.time()
    log.header("Step 9/11: Preprocessing XML files in parallel")
    xml_files = []
    for root, _, files in os.walk(os.path.join(decompile_dir, 'resources')):
        for file in files:
            if file.endswith('.xml'):
                xml_files.append(os.path.join(root, file))

    if xml_files:
        with Pool() as pool:
            results = pool.map(process_xml_file, xml_files)
        if all(results):
            log.success("All XML files preprocessed successfully")
        else:
            log.warning("Some XML files failed to preprocess")
    else:
        log.info("No XML files found for preprocessing")
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    # Step 10: Build APK
    start_time = time.time()
    log.header("Step 10/11: Building modified APK")
    if os.path.exists("out.apk"):
        os.remove("out.apk")

    run_with_spinner(
        f'java -jar APKEditor-1.4.3.jar b -i "{decompile_dir}" -o out.apk',
        spinner_message="Building APK"
    )
    if os.path.exists("out.apk"):
        out_size = os.path.getsize("out.apk") / (1024 * 1024)
        log.success(f"Build completed in {time.time() - start_time:.1f} seconds (Size: {out_size:.2f} MB)")
    else:
        log.error("Failed to build modified APK")
        return None
    progress.update(1)

    # Step 11: Sign the APK
    start_time = time.time()
    log.header("Step 11/11: Signing the APK")
    run_with_spinner(
        "java -jar uber-apk-signer.jar -a out.apk --overwrite",
        spinner_message="Signing APK"
    )

    signed_apk_patterns = [
        "out-aligned-signed.apk",
        "out-signed.apk",
        "out-debugSigned.apk",
        "out-aligned-debugSigned.apk"
    ]

    signed_apk = None
    for pattern in signed_apk_patterns:
        if os.path.exists(pattern):
            signed_apk = pattern
            break

    output_name = f"{base_name}-patched.apk"

    if signed_apk:
        shutil.move(signed_apk, output_name)
        sign_size = os.path.getsize(output_name) / (1024 * 1024)
        log.success(f"APK signed successfully (Size: {sign_size:.2f} MB)")
    else:
        if os.path.exists("out.apk"):
            shutil.move("out.apk", output_name)
            log.warning(f"Signing failed. Using unsigned APK: {output_name}")
        else:
            log.error("Error: No output APK found")
            return None
    log.success(f"Completed in {time.time() - start_time:.1f} seconds")
    progress.update(1)

    log.close_progress("main")
    return output_name

def main():
    """Main function"""
    print(f"\n{Style.BRIGHT}{Fore.CYAN}{'═' * 50}{Style.RESET_ALL}")
    print(f"{Style.BRIGHT}{Fore.CYAN}PairIP Protection Remover v1.3 (Colab Optimized){Style.RESET_ALL}")
    print(f"{Style.BRIGHT}{Fore.CYAN}Please upload your .apks file{Style.RESET_ALL}")
    print(f"{Style.BRIGHT}{Fore.CYAN}{'═' * 50}{Style.RESET_ALL}\n")

    uploaded = files.upload()
    apks_file = next((f for f in uploaded.keys() if f.endswith('.apks')), None)

    if not apks_file:
        log.error("No .apks file uploaded")
        return

    try:
        with zipfile.ZipFile(apks_file, 'r') as zip_ref:
            zip_ref.testzip()
        log.success(f"Input file '{apks_file}' is a valid ZIP archive")
    except zipfile.BadZipFile:
        log.error(f"Input file '{apks_file}' is not a valid .apks file")
        return

    start_time = time.time()
    log.info(f"Processing file: {os.path.basename(apks_file)}")
    out_name = process_apk(apks_file)

    if out_name:
        output_path = os.path.abspath(out_name)
        output_size = os.path.getsize(output_path) / (1024 * 1024)

        # Clean up temporary files
        log.header("Cleaning up temporary files...")
        cleanup_count = 0
        for f in ["base.apk", "merged_app.apk", "out.apk", "merged_app_decompile_xml"] + glob.glob("*.tmp*") + glob.glob("tmp-*"):
            if delete_dir_crossplatform(f):
                cleanup_count += 1
        log.success(f"Removed {cleanup_count} temporary files/directories")

        print(f"\n{Style.BRIGHT}{Fore.GREEN}╔══════════════════════════════════════════════╗{Style.RESET_ALL}")
        print(f"{Style.BRIGHT}{Fore.GREEN}║              PROCESS COMPLETE                ║{Style.RESET_ALL}")
        print(f"{Style.BRIGHT}{Fore.GREEN}╚══════════════════════════════════════════════╝{Style.RESET_ALL}")
        print(f"\n{Fore.GREEN}✓ Final APK: {Style.BRIGHT}{out_name}{Style.RESET_ALL}")
        print(f"{Fore.GREEN}✓ Size: {output_size:.2f} MB{Style.RESET_ALL}")
        print(f"{Fore.GREEN}✓ Location: {output_path}{Style.RESET_ALL}")
        print(f"{Fore.GREEN}✓ Total processing time: {time.time() - start_time:.1f} seconds{Style.RESET_ALL}")
        print(f"{Fore.GREEN}✓ Downloading patched APK...{Style.RESET_ALL}")
        files.download(output_path)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print(f"\n{Fore.YELLOW}Process cancelled by user.{Style.RESET_ALL}")
        sys.exit(0)
    except Exception as e:
        print(f"\n{Fore.RED}An unexpected error occurred: {e}{Style.RESET_ALL}")
        sys.exit(1)


══════════════════════════════════════════════════
PairIP Protection Remover v1.3 (Colab Optimized)
Please upload your .apks file
══════════════════════════════════════════════════



Saving iMe_11.9.5_Premium.ModderSU.COM.apk to iMe_11.9.5_Premium.ModderSU.COM.apk
[✗] No .apks file uploaded
